<a href="https://colab.research.google.com/github/moung1012/Numpy-Pandas/blob/master/keras5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 합성곱 신경망 소개
- 컨브넷(convnet)이라 불리우는 합성곱 신경망(convolutional neural network)소개
- 이 딥러닝 모델은 대부분 컴퓨터 비전 애플리케이션에 사용
- 작은 훈련 데이터셋을 사용한 이미지 분류 문제에 컨브넷 적용하는 사례


In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#컨브넷은 (image_height, image_width, iamage_channels) 크기의 입력 텐서 사용
#이 예제는 MNIST 이미지 포맷인 (28, 28, 1)크기의 입력을 처리하도록 컨브넷을 설정해야 한다
#이 때문에 매개변수로 input_shape=(28, 28, 1)전달

In [ ]:
model.summary()

- Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels)크기의 3D 텐서
- 높이와 넓이 차원은 네트워크가 깊어 질수록 작아지는 경향이 있다
- 채널의 수는 Conv2D 층에 전달된 첫 번째 매개변수에 의해 조절(32개 또는 64개)
- 다음 단계에서 마지막 층의(3, 3, 64)출력 텐서를 완전 연결 네트워크에 주입
- 이 네트워크는 Dense층을 쌓는 분류기: 이 분류기는 1D 벡터를 처리하는데 이전 층의 출력이 3D 텐서이다
- 먼저 3D 출력을 1D 텐서로 펼쳐야 한다.
- 그 다음 Dense층을 추가



In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
# 10개의 클래스를 분류하기 위해 마지막 층을 출력 크기를 10으로 하고 소프트맥스 활성화 함수를 사용한다

In [ ]:
model.summary()


(3, 3, 64)의 3D 텐서의  출력이 (576,)의 1D 벡터 크기로 펼처진후 Dense층으로 주입  
flatten (Flatten)            (None, 576)               0           
MNIST 숫자 이미지에 이 컨브넷을 훈련


In [ ]:
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
test_acc

기존의 완전 연결 네트워크보다 더 높은 테스트 정확도를 컴브넷이 얻었다  
Conv2D와 MaxPolling2D층이 어떤 일을 하는지 살펴보자

---
### 합성곱 연산
- 완전 연결 층과 합성곱 층 사이의 근본적인 차이
- Dense 층은 입력 특성 공간에 있는 전역 패턴(MNIST 숫자 이미지에서는 모든 픽셀에 거친 패턴)을 학습
- 합성곱 층은 지역 패턴을 학습한다
  - 이미지일 경우 작은 2D 윈도우로 입력에서 패턴을 찾는다(앞의 예시는 3 X 3크기)
---

**이 핵심 특징은 컨브넷의 두 가지 흥미로운 성질을 제공**
1. 학습된 패턴은 평행 이동 불변성(translation invariant)을 가진다
  - 컨브넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳(EX: 왼쪽 위 모서리)에서도 이 패턴을 인식할 수 있다
  - 완전 연결 네트워크는 새로운 위치에 나타난 것은 새로운 패턴으로 학습해야 한다
  - 이 성질로 인해 적은 수의 훈련 샘플을 사용하더라도 일반화 능력을 가진 표현을 학습할 수 있다
2. 컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있다
  - 첫 번째 합성곱 층이 에지 같은 작은 지역 패턴을 학습
  - 두 번째 합성곱 층은 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습하는 식이다
  - 이런 방식을 사용해 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있다
  


---
합성곱 연산은 **특성 맵(Feature map)** 이라고 부르는 3D 텐서에 적용된다  
**특성 맵**  
- 이 텐서는 2개의 공간 축(height, width)과 깊이 축(channel 축이라고도 한다)으로 구성
  - RGB 이미지는 3개의 컬러 채널(빨,녹,파)을 가지므로 깊이 축의 차원이 3이 된다
  - MNIST 숫자처럼 흑백 이미지는 깊이 축의 차원이 1(회색 톤)
---
합성곱 연산은 입력 특성 맵에서 작은 패치(patch)들을 추출  
이런 모든 패치에 같은 변환을 작용하여 출력 특성 맵(output feature map)을 만듦  
- 출력 특성 맵도 높이와 너비를 가진 3D 텐서입니다
- 출력 텐서의 깊이는 층의 매개변수로 결정되기 때문에 상황에 따라 다르다
- 깊이 축의 채널은 더이상 컬러를 의미하는게 아닌 일종의 필터를 의미한다
- 필터는 입력 데이터의 어떤 특성을 인코딩
  - EX) 고 수준으로 보면 하나의 필터가 '입력에 얼굴이 있는지'를 인코딩할 수 있다
---
특성 맵이란 말의 의미  
- 깊이 축에 있는 각 차원은 하나의 특성
- 2D텐서 output[ : , : , n]은 입력에 대한 이 필터 응답을 나타내는 2D 공간상의 맵
---
합성곱은 핵심적인 2개의 파라미터로 정의
1. 입력으로부터 뽑아낼 패치의 크기: 전형적으로 3 X 3 또는 5 X 5크기를 사용
2. 특성 맵의 출력 깊이: 합성곱으로 계산할 필터의 수
---
케라스의 Conv2D층에서 이 파라미터는 Conv2D(output_deth,(window_height,window_width))처럼 첫 번째와 두 번째 매개변수로 전달된다
    
- 3D 입력 특성 맵 위를 3X3 또는 5X5 크기의 윈도우가 슬라이딩하면서 모든 위치에서 3D 특성 패치 ((window_height, window_width, input_depth)크기)를 추출하는 방식으로 합성곱이 작동  
- 이런 3D 패치는 (output_depth,) 크기의 1D 벡터로 변환  
- 변환된 모든 벡터는 (height, width, output_depth) 크기의 3D특성 맵으로 재구성된다
---
출력 높이와 너비는 입력의 높이, 너비와 다를 수 있다
1. 경계 문제. 입력 특성 맵에 패딩을 추가하여 대응가능
2. 스트라이드(stride)의 사용 여부
---




### 경계 문제와 패딩 이해하기
- 패딩(padding)은 입력 특성 맵의 가장자리에 적절한 개수와 행과 열을 추가한다
- 모든 입력 타일에 합성곱 윈도우의 중앙을 위치시킬 수 있다
- Conv2D 층에서 패딩은 매개변수로 설정할 수 있다
  - 2개의 값이 가능
  1. valid : 패딩을 사용하지 않는다는 뜻
  2. same : 입력과 동일한 높이와 너비를 가진 출력을 만들기 위해 패딩한다 라는뜻
  - 패딩의 매개변수의 기본 값은 valid

---
###합성곱 스트라이드 이해하기
출력 크기에 영향을 미치는 다른 요소는 스트라이드
- 두 번의 연속적인 윈도우 사이의 거리가 스트라이드라고 불리는 합성곱 파라미터
- 스트라이드의 기본값은 1 스트라이드가 1보다 큰 스트라이드 합성곱도 가능
- 스트라이드는 실전에서 드물게 사용되지만 어떤 모델에서 유용하게 사용될 수 있으므로 알아 둘 필요가 있다
- 특성 맵을 다운샘플링하기 위해서 스트라이드 대신에 첫 번째 컨브넷 예제에 사용된 최대 풀링 연산을 사용하는 경우가 많다



### 최대 풀링 연산
- 최대 풀링은 입력 특성 맵에서 윈도우에 맞는 패치를 추출하고 각 채널 별로 최댓값을 출력
- 합성곱과 개념적으로 비슷하지만 추출한 패치에 학습된 선형 변환(합성곱 커널)을 작용하는 대신 하드 코딩된 최댓값 추출 연산을 사용한다
- 합성곱과 가장 큰 차이점은 최대 풀링은 보통 2X2윈도우와 스트라이드 2를 사용하여 특성맵을 절반 크기로 다운샘플링 한다는것
- 이에 반해 합성곱은 전형적으로 3X3 윈도우와 스트라이드 1을 사용한다



In [ ]:
from keras import models
from keras import layers
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32,(3,3), activation='relu',
                                    input_shape=(28,28,1)))
model_no_max_pool.add(layers.Conv2D(64,(3,3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64,(3,3), activation='relu'))

In [ ]:
model_no_max_pool.summary()

이 설정의 2가지 문제
- 특성의 공간적 계층 구조를 학습하는 데 도움이 되지 않는다
  - 세 번째 층의 3X3 윈도우는 초기 입력의 7X7윈도우 영역에 대한 정보만 담고있다 -> 컨브넷에 의해 학습된 고 수준 패턴은 초기 입력에 관한 정보가 아주 적어 숫자 분류 학습을 하기에 충분하지 않을 것이다
  - 마지막 합성곱 층의 특성이 전체 입력에 대한 정보를 가지고 있어야 한다
- 최종 특성 맵은 22 X 22 X 64 = 36928의 원소를 갖는다
  - 이 컨브넷을 펼친 후 512 크기의 Dense층과 연결한다면 15.8백만 개의 가중치 파라미터가 생긴다-> 작은 모델 치고는 너무 많은 가중치고, 심각한 과대적합이 발생할 것이다  

다운샘플링을 사용하는 이유는 처리할 특성 맵의 가중치 개수를 줄이고 연속적인 합성 곱 층이 점점 커진 윈도우를 통해 바라보도록 만들어 필터의 공간적인 계층 구조를 구성한다
